In [1]:
# after http://r2rt.com/recurrent-neural-networks-in-tensorflow-ii.html
"""
Imports
"""
import numpy as np
import tensorflow as tf
%matplotlib inline
import matplotlib.pyplot as plt
import time
import os
import urllib.request
from tensorflow.models.rnn.ptb import reader

In [2]:
def gen_epochs(n, num_steps, batch_size):
    for i in range(n):
        yield reader.ptb_iterator(data, batch_size, num_steps)

def reset_graph():
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()


In [3]:
file_name = '/home/key/Downloads/linux-4.9-rc7/kernel/all.c'

with open(file_name,'r') as f:
    raw_data = f.read()
    vocab = set(raw_data)
    
idx_to_vocab = dict(enumerate(vocab))
vocab_to_idx = dict(zip(idx_to_vocab.values(), idx_to_vocab.keys()))
vocab_size = len(vocab)

data = [vocab_to_idx[c] for c in raw_data]
print(raw_data[:100])
print(vocab)
print(idx_to_vocab)
del raw_data


/* Copyright (c) 2011-2014 PLUMgrid, http://plumgrid.com
 *
 * This program is free software; you ca
{'j', 'w', 'D', '$', '_', '0', '>', 'l', ' ', 'c', 'å', ':', 'q', 'F', 'v', ',', '5', 'J', '=', 'T', 'B', '8', '&', 'S', '9', ';', '[', 'a', '<', 'm', '!', '`', '4', 'U', 'G', '~', '6', 'R', 'X', 'A', 'Z', 'h', '\n', '?', '^', 'E', 'f', 'H', '*', 'b', 'z', 'n', 'Q', 'r', 'e', 'o', 'x', '+', '#', 'g', '%', '.', '(', 'y', 'L', 'O', '}', '3', '-', '1', 'p', 'P', 'V', "'", '\t', '"', '\\', 'W', 'k', '{', '@', 'N', 'Y', 'I', '|', ')', '/', '7', 'M', 'u', 'C', 'd', ']', 'i', '2', 'K', 't', 's'}
{0: 'j', 1: 'w', 2: 'D', 3: '$', 4: '_', 5: '0', 6: '>', 7: 'l', 8: ' ', 9: 'c', 10: 'å', 11: ':', 12: 'q', 13: 'F', 14: 'v', 15: ',', 16: '5', 17: 'J', 18: '=', 19: 'T', 20: 'B', 21: '8', 22: '&', 23: 'S', 24: '9', 25: ';', 26: '[', 27: 'a', 28: '<', 29: 'm', 30: '!', 31: '`', 32: '4', 33: 'U', 34: 'G', 35: '~', 36: '6', 37: 'R', 38: 'X', 39: 'A', 40: 'Z', 41: 'h', 42: '\n', 43: '?', 44: '^', 45: 'E',

In [4]:
def train_network(g, num_epochs, num_steps = 120, batch_size = 32, verbose = True, save=False):
    tf.set_random_seed(2345)
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        training_losses = []
        for idx, epoch in enumerate(gen_epochs(num_epochs, num_steps, batch_size)):
            training_loss = 0
            steps = 0
            training_state = None
            for X, Y in epoch:
                steps += 1

                feed_dict={g['x']: X, g['y']: Y}
                if training_state is not None:
                    feed_dict[g['init_state']] = training_state
                training_loss_, training_state, _ = sess.run([g['total_loss'],
                                                      g['final_state'],
                                                      g['train_step']],
                                                             feed_dict)
                training_loss += training_loss_
            if verbose:
                print("Average training loss for Epoch", idx, ":", training_loss/steps)
            training_losses.append(training_loss/steps)

        if isinstance(save, str):
            g['saver'].save(sess, save)

    return training_losses

In [5]:
def build_graph(
    cell_type = None,
    num_weights_for_custom_cell = 5,
    state_size = 128,
    num_classes = vocab_size,
    batch_size = 32,
    num_steps = 120,
    num_layers = 3,
    build_with_dropout=False,
    learning_rate = 1e-4):

    reset_graph()

    x = tf.placeholder(tf.int32, [batch_size, num_steps], name='input_placeholder')
    y = tf.placeholder(tf.int32, [batch_size, num_steps], name='labels_placeholder')

    dropout = tf.constant(1.0)

    embeddings = tf.get_variable('embedding_matrix', [num_classes, state_size])

    rnn_inputs = tf.nn.embedding_lookup(embeddings, x)

    if cell_type == 'Custom':
        cell = CustomCell(state_size, num_weights_for_custom_cell)
    elif cell_type == 'GRU':
        cell = tf.nn.rnn_cell.GRUCell(state_size)
    elif cell_type == 'LSTM':
        cell = tf.nn.rnn_cell.LSTMCell(state_size, state_is_tuple=True)
    elif cell_type == 'LN_LSTM':
        cell = LayerNormalizedLSTMCell(state_size)
    else:
        cell = tf.nn.rnn_cell.BasicRNNCell(state_size)

    if build_with_dropout:
        cell = tf.nn.rnn_cell.DropoutWrapper(cell, input_keep_prob=dropout)

    if cell_type == 'LSTM' or cell_type == 'LN_LSTM':
        cell = tf.nn.rnn_cell.MultiRNNCell([cell] * num_layers, state_is_tuple=True)
    else:
        cell = tf.nn.rnn_cell.MultiRNNCell([cell] * num_layers)

    if build_with_dropout:
        cell = tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=dropout)

    init_state = cell.zero_state(batch_size, tf.float32)
    rnn_outputs, final_state = tf.nn.dynamic_rnn(cell, rnn_inputs, initial_state=init_state)

    with tf.variable_scope('softmax'):
        W = tf.get_variable('W', [state_size, num_classes])
        b = tf.get_variable('b', [num_classes], initializer=tf.constant_initializer(0.0))

    #reshape rnn_outputs and y
    rnn_outputs = tf.reshape(rnn_outputs, [-1, state_size])
    y_reshaped = tf.reshape(y, [-1])

    logits = tf.matmul(rnn_outputs, W) + b

    predictions = tf.nn.softmax(logits)

    total_loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits, y_reshaped))
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(total_loss)

    return dict(
        x = x,
        y = y,
        init_state = init_state,
        final_state = final_state,
        total_loss = total_loss,
        train_step = train_step,
        preds = predictions,
        saver = tf.train.Saver()
    )

In [6]:
def ln(tensor, scope = None, epsilon = 1e-5):
    """ Layer normalizes a 2D tensor along its second axis """
    assert(len(tensor.get_shape()) == 2)
    m, v = tf.nn.moments(tensor, [1], keep_dims=True)
    if not isinstance(scope, str):
        scope = ''
    with tf.variable_scope(scope + 'layer_norm'):
        scale = tf.get_variable('scale',
                                shape=[tensor.get_shape()[1]],
                                initializer=tf.constant_initializer(1))
        shift = tf.get_variable('shift',
                                shape=[tensor.get_shape()[1]],
                                initializer=tf.constant_initializer(0))
    LN_initial = (tensor - m) / tf.sqrt(v + epsilon)

    return LN_initial * scale + shift

In [7]:
class LayerNormalizedLSTMCell(tf.nn.rnn_cell.RNNCell):
    """
    Adapted from TF's BasicLSTMCell to use Layer Normalization.
    Note that state_is_tuple is always True.
    """

    def __init__(self, num_units, forget_bias=1.0, activation=tf.nn.tanh):
        self._num_units = num_units
        self._forget_bias = forget_bias
        self._activation = activation

    @property
    def state_size(self):
        return tf.nn.rnn_cell.LSTMStateTuple(self._num_units, self._num_units)

    @property
    def output_size(self):
        return self._num_units

    def __call__(self, inputs, state, scope=None):
        """Long short-term memory cell (LSTM)."""
        with tf.variable_scope(scope or type(self).__name__):
            c, h = state

            # change bias argument to False since LN will add bias via shift
            concat = tf.nn.rnn_cell._linear([inputs, h], 4 * self._num_units, False)

            i, j, f, o = tf.split(1, 4, concat)

            # add layer normalization to each gate
            i = ln(i, scope = 'i/')
            j = ln(j, scope = 'j/')
            f = ln(f, scope = 'f/')
            o = ln(o, scope = 'o/')

            new_c = (c * tf.nn.sigmoid(f + self._forget_bias) + tf.nn.sigmoid(i) *
                   self._activation(j))

            # add layer_normalization in calculation of new hidden state
            new_h = self._activation(ln(new_c, scope = 'new_h/')) * tf.nn.sigmoid(o)
            new_state = tf.nn.rnn_cell.LSTMStateTuple(new_c, new_h)

            return new_h, new_state

In [8]:
g1 = build_graph(cell_type='GRU', num_steps=120)
t = time.time()
losses = train_network(g1, 20, num_steps=120, save="models/GRU_128_120")
print("It took", time.time() - t, "seconds to train for 20 epochs.")
print("The average loss on the final epoch was:", losses[-1])

Average training loss for Epoch 0 : 3.10970336368
Average training loss for Epoch 1 : 2.34120994661
Average training loss for Epoch 2 : 2.02586186176
Average training loss for Epoch 3 : 1.83284370382
Average training loss for Epoch 4 : 1.70492588829
Average training loss for Epoch 5 : 1.6128305805
Average training loss for Epoch 6 : 1.54402266864
Average training loss for Epoch 7 : 1.49071158275
Average training loss for Epoch 8 : 1.44772524777
Average training loss for Epoch 9 : 1.41203714179
Average training loss for Epoch 10 : 1.38177386954
Average training loss for Epoch 11 : 1.35571193106
Average training loss for Epoch 12 : 1.33297513603
Average training loss for Epoch 13 : 1.31288076414
Average training loss for Epoch 14 : 1.2949077397
Average training loss for Epoch 15 : 1.27867760804
Average training loss for Epoch 16 : 1.26390564819
Average training loss for Epoch 17 : 1.25036272306
Average training loss for Epoch 18 : 1.23785472765
Average training loss for Epoch 19 : 1.2262

In [9]:
"""
g = build_graph(cell_type='LSTM', num_steps=80)
t = time.time()
losses = train_network(g, 20, num_steps=80, save="models/LSTM_20_epochs")
print("It took", time.time() - t, "seconds to train for 20 epochs.")
print("The average loss on the final epoch was:", losses[-1])
"""

'\ng = build_graph(cell_type=\'LSTM\', num_steps=80)\nt = time.time()\nlosses = train_network(g, 20, num_steps=80, save="models/LSTM_20_epochs")\nprint("It took", time.time() - t, "seconds to train for 20 epochs.")\nprint("The average loss on the final epoch was:", losses[-1])\n'

In [10]:
def generate_characters(g, checkpoint, num_chars, prompt='A', pick_top_chars=None):
    """ Accepts a current character, initial state"""

    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        g['saver'].restore(sess, checkpoint)

        state = None
        current_char = vocab_to_idx[prompt]
        chars = [current_char]

        for i in range(num_chars):
            if state is not None:
                feed_dict={g['x']: [[current_char]], g['init_state']: state}
            else:
                feed_dict={g['x']: [[current_char]]}

            preds, state = sess.run([g['preds'],g['final_state']], feed_dict)

            if pick_top_chars is not None:
                p = np.squeeze(preds)
                p[np.argsort(p)[:-pick_top_chars]] = 0
                p = p / np.sum(p)
                current_char = np.random.choice(vocab_size, 1, p=p)[0]
            else:
                current_char = np.random.choice(vocab_size, 1, p=np.squeeze(preds))[0]

            chars.append(current_char)

    chars = map(lambda x: idx_to_vocab[x], chars)
    print("".join(chars))
    return("".join(chars))



In [11]:
g3 = build_graph(cell_type='GRU', num_steps=1, batch_size=1)
generate_characters(g3, "models/GRU_128_120", 750, prompt='/')

/QSOWANTBULTU_TRRYRASH_SYSZP]
	 */
	write_buffer_entries_lock();

	if (audit->load->flags & cmp_mad ||
	    tathcomp_finishet_put(sec, dop, next)) {
		test:					      profile->perf_all && nr_call->init, 0);
		current->lock = 0; i < so = and;
	case TIDE::
		ret = sdltxtrab(0);
		count = info->hb_to_ns[ivers);
		if (pd_threads) {
			for_to_next_off(comproup->curr_ecps);
			if (_j->cpu_szaute <= NULL ||
			 * info fields[iter->function.dlistvalls_current) {
				raw_read = wait_log_can_err_free_cpu(vmop_new_log_lal, *ullaglec", 3duwnabacktrizer);
				if (p->next_t) {
					if (!(info->per_cpumask)
					case freeting_tasks.handler[PM_SIQDE);
				const bsesscture_resource;
				continue2_printk("vorkering.tvstark }tmap %s slccmp, *intersnon *offset,


''